# KorBERT End to End
Author: MyungHoon Jin

Import Library

In [1]:
import collections
import re
import unicodedata
import six
import csv
import os
import copy
import json
import math
import numpy as np
import pandas as pd
from eunjeon import Mecab
from konlpy.tag import Kkma, Okt
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Data Preparing
한국어 BERT를 실습하기 위해 준비해야할 사항들이 있다.

우선, pre-training BERT를 적용하기 위해 Dacon2019 금융문자 스미싱 데이터를 활용하여 이진분류 문제를 풀어보겠다.

데이터는 오른쪽 링크에서 얻을 수 있다. https://dacon.io/cpt14

In [2]:
data_path = './data/' # 각자의 data path를 입력하도록 하자.
df_train = pd.read_csv(data_path + 'train.csv')
df_test = pd.read_csv(data_path + 'public_test.csv')

In [3]:
df_train.head()

,id,year_month,text,smishing
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0
2,2,2017-01,안녕하십니까 고객님. XXX은행입니다.금일 납부하셔야 할 금액은 153600원 입니...,0
3,4,2017-01,XXX 고객님안녕하세요XXX은행 XXX지점입니다지난 한 해 동안 저희 XXX지점에 ...,0
4,5,2017-01,1월은 새로움이 가득XXX입니다.올 한해 더 많이행복한 한해되시길바랍니다,0


In [4]:
df_test.head()

,id,year_month,text
0,340000,2019-01,XXX고객님! 안녕하세요? 새롭게 시작하는 한 주 행복 가득하시길 기원합니다. 지난...
1,340001,2019-01,긴급 안내 XXX은행 가락동 지점 - 헬리오XXX 기본XXX 대출이자를 ...
2,340002,2019-01,XXX 고객님 안녕하세요올해는 미세먼지가 유난인거 같습니다.엊그제 새해가 시작된거같...
3,340003,2019-01,XXX 고객님찾아온 행운을 잡으셨나요? 못잡으셨다면 이번에 다시 잡으시길 기원합니다...
4,340004,2019-01,XXX 고객님새해 복 많이 받으세요 XXX은행 코스트코 퇴직연금 담당자입니다. 고...


BERT Input을 위해 가벼운(그러나 오래걸리는) 전처리를 실시해보도록 하자.

In [5]:
# spacing
from chatspace import ChatSpace

spacer = ChatSpace()

UnicodeDecodeError: 'cp949' codec can't decode byte 0xec in position 22: illegal multibyte sequence

위와 같은 에러가 발생해도 당황하지 마라. (Linux, Mac을 이용하는 유저는 위의 에러가 뜨지 않을 것이다.)

Window 유저만 해당하는 cp949문제이다.

기존 PINGPONG에서 제공하는 API에서는 해당 encoding에 접근할 parameter가 없기 때문에 이를 수정하여 사용하도록 하자.

우선, python shell에서 다음의 명령어를 입력하여 자신의 envs의 site-packages가 어디있는지 확인하라.

In [6]:
import site

site.getsitepackages()

['C:\\Users\\jinma\\Anaconda3\\envs\\basic',
 'C:\\Users\\jinma\\Anaconda3\\envs\\basic\\lib\\site-packages']

다음으로, 해당 site-packages에서 chatspace/inference.py 파일에 들어가서 아래 사항들을 수정하라.

```python
# first, revise function '_load_vocab' lines as follow; (line 196~207)
    def _load_vocab(self, vocab_path: str, encoding: str) -> Vocab:
        """
        저장된 vocab 을 로딩

        :param vocab_path: vocab 위치
        :return: 로딩된 vocab
        """
        with open(vocab_path, encoding=encoding) as f:
            vocab_tokens = [line.strip() for line in f]
        vocab = Vocab(tokens=vocab_tokens)
        self.config["vocab_size"] = len(vocab)
        return vocab
    
# And then, revise __init__ as follow; (line 31~50)
class ChatSpace:
    def __init__(
        self,
        model_path: str = None,
        config_path: str = CONFIG_PATH,
        vocab_path: str = VOCAB_PATH,
        device: str = "cpu",
        from_jit: bool = True,
        encoding: str = 'utf-8',
    ):
        self.config = self._load_config(config_path)
        self.vocab = self._load_vocab(vocab_path, encoding=encoding)
        self.device = torch.device(device)

        if model_path is None:
            from_jit = self._is_jit_available() if from_jit else False
            model_path = JIT_MODEL_PATH if from_jit else MODEL_DICT_PATH

        self.model = self._load_model(model_path, self.device, from_jit=from_jit)
        self.model.eval()
```

그러면 완성☆ (jupyter notebook을 이용할 경우 kernel을 죽이고 재시동해야 된다.)

In [39]:
# spacing
from chatspace import ChatSpace

# If you want to use gpu on torch, set parameter name 'device' as 'cuda'.
spacer = ChatSpace(device='cuda')

Loading JIT Compiled ChatSpace Model


PINGPONG에서 제공하는 띄어쓰기 API를 적용해보자.

In [42]:
RESOURCE_PATH = 'C:/Users/jinma/Anaconda3/envs/basic/Lib/site-packages/chatspace'

VOCAB_PATH = os.path.join(RESOURCE_PATH, "vocab.txt")
MODEL_DICT_PATH = os.path.join(RESOURCE_PATH, "model/model.pt")
JIT_MODEL_PATH = os.path.join(RESOURCE_PATH, "model/model_jit.pt")
CONFIG_PATH = os.path.join(RESOURCE_PATH, "config.json")

In [44]:
spacer.config

{'learning_rate': 0.001,
 'batch_size': 512,
 'valid_ratio': 0.025,
 'epochs': 15,
 'vocab_min_freq': 3,
 'min_seq_len': 7,
 'num_workers': 0,
 'logging_step': 500,
 'validation_step': 1500,
 'test_threshold': 0.5,
 'use_multi_gpu': False,
 'device': 'cuda',
 'embedding_dim': 256,
 'cnn_filter': 3,
 'cnn_features': 128,
 'space_prob': [0, 0.15, 0.4],
 'lstm_bidirectional': True,
 'dropout_keep_prob': 0.1,
 'apply_weight': False,
 'lstm_layers': 1,
 'vocab_size': 3563}

In [45]:
spacer.vocab

{'[PAD]': 0,
 '[UNK]': 1,
 '[SOS]': 2,
 '[EOS]': 3,
 ' ': 4,
 '<pad>': 5,
 '<unk>': 6,
 '<space>': 7,
 '.': 8,
 '이': 9,
 '어': 10,
 '아': 11,
 '?': 12,
 '다': 13,
 '나': 14,
 '가': 15,
 '그': 16,
 '요': 17,
 'ㅋ': 18,
 '고': 19,
 '지': 20,
 '는': 21,
 '하': 22,
 '니': 23,
 '도': 24,
 '응': 25,
 'ㅎ': 26,
 '자': 27,
 '거': 28,
 '해': 29,
 '에': 30,
 '야': 31,
 '게': 32,
 '서': 33,
 '안': 34,
 '오': 35,
 '데': 36,
 '기': 37,
 'ㅠ': 38,
 '내': 39,
 '있': 40,
 '리': 41,
 '!': 42,
 '시': 43,
 '은': 44,
 '사': 45,
 '네': 46,
 '한': 47,
 '라': 48,
 '일': 49,
 '제': 50,
 '래': 51,
 '구': 52,
 '면': 53,
 '보': 54,
 ',': 55,
 '마': 56,
 '잘': 57,
 '을': 58,
 '만': 59,
 '너': 60,
 '까': 61,
 '먹': 62,
 '뭐': 63,
 '말': 64,
 '왜': 65,
 '난': 66,
 '들': 67,
 '로': 68,
 '무': 69,
 '으': 70,
 '웅': 71,
 '여': 72,
 '저': 73,
 '좀': 74,
 '우': 75,
 '금': 76,
 '수': 77,
 '대': 78,
 '없': 79,
 '좋': 80,
 '겠': 81,
 '주': 82,
 '진': 83,
 '러': 84,
 '와': 85,
 '더': 86,
 '했': 87,
 '할': 88,
 '정': 89,
 '같': 90,
 '알': 91,
 '집': 92,
 '려': 93,
 '음': 94,
 '전': 95,
 '인': 96,
 '럼': 97,


In [54]:
import sys
sys.path.append(RESOURCE_PATH)
sys.path.append(RESOURCE_PATH + '/data')
sys.path.append(RESOURCE_PATH + '/data/corpus')
sys.path.append(RESOURCE_PATH + '/data/indexer')
sys.path.append(RESOURCE_PATH + '/data/vocab')

In [58]:
from corpus import DynamicCorpus
from indexer import Indexer
from vocab import Vocab

ImportError: attempted relative import with no known parent package

In [53]:
from dataset import ChatSpaceDataset

ImportError: attempted relative import with no known parent package

In [32]:
text = texts.iloc[0]

In [35]:
if isinstance(text, str):
    batch_texts = [text]

In [37]:
torch.cuda.is_available()

True

In [38]:
torch.cuda.get_device_name(0)

'GeForce RTX 2060'

In [36]:
for batch_texts

['XXX은행성산XXX팀장입니다.행복한주말되세요']

In [26]:
# train과 test length 기록
len_train, len_test = len(df_train), len(df_test)

# train과 test의 text를 concatenate
texts = pd.concat((df_train.text, df_test.text))

# Spacing
%time texts = list(map(lambda x: spacer.space(x), texts.values))

RuntimeError: Expected object of backend CUDA but got backend CPU for argument #3 'index'
The above operation failed in interpreter, with the following stack trace:
at code/model_jit.py:43:17
  bias1 = _22.bias
  _23 = self.batch_normalization
  weight3 = _23.weight
  bias2 = _23.bias
  running_mean = _23.running_mean
  running_var = _23.running_var
  _24 = self.layer_normalization
  weight4 = _24.weight
  bias3 = _24.bias
  embed_input = torch.embedding(weight, input, -1, False, False)
                ~~~~~~~~~~~~~~~ <--- HERE
  input0 = torch.transpose(embed_input, 1, 2)
  input1 = torch._convolution(input0, _2, _3, [1], [0], [1], False, [0], 1, False, False, True)
  input2 = torch.constant_pad_nd(input1, [1, 1], 0)
  input3 = torch._convolution(input2, _5, _6, [1], [0], [1], False, [0], 1, False, False, True)
  conv2_paded = torch.constant_pad_nd(input3, [3, 3], 0)
  input4 = torch.cat([input2, conv2_paded], 1)
  input5 = torch._convolution(input4, _8, _9, [1], [0], [1], False, [0],